# Formant Frequencies

# Measuring Formants

## Import modules

In [27]:
import parselmouth
from parselmouth.praat import call
from glob import glob
import os
import pandas as pd

## Create list of files

In [28]:
file_list = sorted(glob("/home/david/Sharing/fromant_check_for_averageness/new_voices/new_voices/*.wav"))
basename_list = [os.path.splitext(os.path.basename(filename))[0] for filename in file_list]
basename_list[:5]

['f1745_vowels1_10_u',
 'f1745_vowels1_2_a',
 'f1745_vowels1_4_i',
 'f1745_vowels1_6_e',
 'f1745_vowels1_8_o']

## Load the sounds into Parselmouth-Praat

In [29]:
sounds = [parselmouth.Sound(sound_file) for sound_file in file_list]
sounds[:5]

## Measure the formants

### Analysis parameters matter
- Maximum Formant
    - This is the highest frequency a formant could be
    - Children's voices - 7000 Hz, 8000 Hz
    - Adult voices
        - Lower pitched voices use 5000
        - Higher pitched voices use 5500
    - Our cutoff will be 170 Hz or about the average difference between female and male voice pitch

### `if` the voice pitch is low, use `5000`; `elif` the voice pitch is high, use `5500`

In [30]:
def measure_formants(sound):
    pitch = sound.to_pitch()
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz")
    if mean_pitch < 170:
        max_formant = 5000
    elif mean_pitch >= 170:
        max_formant = 5500
    else:
        max_formant = 5500  # It's always good to have a fallback value just in case
    formants = sound.to_formant_burg(maximum_formant=max_formant)
    t0 = formants.xs()[0]
    t1 = formants.xs()[-1]
    t0_f1 = call(formants, "Get value at time", 1, formants.xs()[0], "Hertz", "Linear")
    t1_f1 = call(formants, "Get value at time", 1, formants.xs()[1], "Hertz", "Linear")
    t0_f2 = call(formants, "Get value at time", 2, formants.xs()[0], "Hertz", "Linear")
    t1_f2 = call(formants, "Get value at time", 2, formants.xs()[1], "Hertz", "Linear")
    t0_f3 = call(formants, "Get value at time", 3, formants.xs()[3], "Hertz", "Linear")
    t1_f3 = call(formants, "Get value at time", 3, formants.xs()[3], "Hertz", "Linear")
    t0_f4 = call(formants, "Get value at time", 4, formants.xs()[4], "Hertz", "Linear")
    t1_f4 = call(formants, "Get value at time", 4, formants.xs()[4], "Hertz", "Linear")
    return t0, t1, t0_f1, t1_f1, t0_f2, t1_f2, t0_f3, t1_f3, t0_f4, t1_f4


## Take the first and last formant value for each file and save the times and values

In [31]:
t0_list = []
t1_list = []
t0_f1_list = []
t1_f1_list = []
t0_f2_list = []
t1_f2_list = []
t0_f3_list = []
t1_f3_list = []
t0_f4_list = []
t1_f4_list = []
for sound in sounds:
    t0, t1, t0_f1, t1_f1, t0_f2, t1_f2, t0_f3, t1_f3, t0_f4, t1_f4 = measure_formants(sound)
    t0_list.append(t0)
    t1_list.append(t1)
    t0_f1_list.append(t0_f1)
    t1_f1_list.append(t1_f1)
    t0_f2_list.append(t0_f2)
    t1_f2_list.append(t1_f2)
    t0_f3_list.append(t0_f3)
    t1_f3_list.append(t1_f3)
    t0_f4_list.append(t0_f4)
    t1_f4_list.append(t1_f4)

## Put Data into Pandas Dataframe

In [32]:
df = pd.DataFrame({
    "filename":file_list,
    "file": basename_list,
    "t0":t0_list,
    "t1":t1_list,
    "t0_f1":t0_f1_list,
    "t1_f1":t1_f1_list,
    "t0_f2":t0_f2_list,
    "t1_f2":t1_f2_list,
    "t0_f3":t0_f3_list,
    "t1_f3":t1_f3_list,
    "t0_f4":t0_f4_list,
    "t1_f4":t1_f4_list,
})
df.head()

,filename,file,t0,t1,t0_f1,t1_f1,t0_f2,t1_f2,t0_f3,t1_f3,t0_f4,t1_f4
0,/home/david/Sharing/fromant_check_for_averagen...,f1745_vowels1_10_u,0.027304,0.264804,438.579902,440.953844,1723.094968,1732.432223,2844.683927,2844.683927,4092.216004,4092.216004
1,/home/david/Sharing/fromant_check_for_averagen...,f1745_vowels1_2_a,0.027188,0.477188,863.545806,1071.143501,1119.496634,1599.744732,3179.910418,3179.910418,4226.145260,4226.145260
2,/home/david/Sharing/fromant_check_for_averagen...,f1745_vowels1_4_i,0.027791,0.396541,403.092767,400.568539,2875.632138,2861.631604,3408.970651,3408.970651,4538.318689,4538.318689
3,/home/david/Sharing/fromant_check_for_averagen...,f1745_vowels1_6_e,0.025540,0.419290,647.188563,658.624367,2463.562471,2341.635741,2649.369488,2649.369488,3151.429867,3151.429867
4,/home/david/Sharing/fromant_check_for_averagen...,f1745_vowels1_8_o,0.026372,0.376372,617.545641,626.335264,1048.296424,1006.562101,2953.308545,2953.308545,3925.155273,3925.155273


## Save Data

In [33]:
df.to_csv('/home/david/Sharing/fromant_check_for_averageness/new_voices/new_voices/formant_measures_straight.csv', index=0)